In [1]:
# First make sure to install all required packages.
# You can do it by running the following command:

In [2]:
# ]add Arrow CSV DataFrames Plots FreqTables StatsBase Chain

In [3]:
# If you launched Jupyter in directory with attached Project.toml and Manifest.toml
# use below command to install required packages with fixed versions. 
# Check Project introduction for more information.

In [4]:
#] instantiate

In [5]:
# Import required libraries
import Downloads
import SHA
using Arrow
using Chain
using CSV
using DataFrames
using Plots
using FreqTables
using Statistics
using StatsBase

# Adult

In [6]:
# Define URL to Adult data and expected SHA1
const ADULT_TRAIN = "https://archive.ics.uci.edu/ml/" *
                    "machine-learning-databases/adult/adult.data"
const ADULT_TRAIN_NAME = "adult_train.txt"

const ADULT_TRAIN_SHA1 = [0xee, 0x86, 0xbb, 0xe5, 0x56,
                          0x57, 0x8f, 0x70, 0x9a, 0xe0,
                          0xfd, 0x00, 0x2a, 0xc5, 0x8a,
                          0xc9, 0x37, 0x26, 0x48, 0x2f];

In [7]:
# Download Adult data if not exists
if isfile(ADULT_TRAIN_NAME)
    @info "$ADULT_TRAIN_NAME found. Skipping download."
else
    @info "$ADULT_TRAIN_NAME not found. Fetching from source."
    Downloads.download(ADULT_TRAIN, ADULT_TRAIN_NAME)
end

┌ Info: adult_train.txt not found. Fetching from source.
└ @ Main In[7]:5


In [ ]:
# Check SHA1 of Adult file
if ADULT_TRAIN_SHA1 == open(SHA.sha1, ADULT_TRAIN_NAME)
    @info "SHA1 check of $ADULT_TRAIN_NAME passed."
else
    error("$ADULT_TRAIN_NAME file has an invalid SHA1. Aborting!")
end

In [ ]:
# We define column names to pass to CSV.read function
const COL_NAMES = [:age, :workclass, :fnlwgt, :education, :education_num,
                   :marital_status, :occupation, :relationship, :race,
                   :sex, :capital_gain, :capital_loss, :hours_per_week,
                   :native_country, :target];

In [ ]:
# Read Adult CSV
adult_train = CSV.read(ADULT_TRAIN_NAME, DataFrame, header=COL_NAMES,
                       delim=", ", missingstring="?", tasks=1)
adult_train_raw = copy(adult_train)

In [ ]:
# Adjust Jupyter's columns output to 200
ENV["COLUMNS"] = 200

In [ ]:
# Adjust Jupyter's rows output to 20
ENV["LINES"] = 20

In [ ]:
# Remove irrelevant fnlwgt feature
select!(adult_train, Not(:fnlwgt))

In [ ]:
# Basic statistics of the dataset
describe(adult_train)

In [ ]:
# To 'catch' all nominal variables we specify Union of String and Missing types
nominal = names(adult_train, Union{String, Missing})

In [ ]:
# This helper function will aggregate each by each category in the column and calculate the counts
# Data is passed to the bar function to create a barplot
function bar_helper(column_name)
    agg = combine(groupby(adult_train, column_name, sort=true), nrow)
    bar(string.(agg[!, column_name]), agg.nrow, xlabel=column_name,
        legend=false, xrotation = 45)
end

In [ ]:
# Inspect distribution of nominal variables
foreach(display∘bar_helper, nominal)

In [ ]:
# Inspect distribution of nominal variables
foreach(name -> println("\n", proptable(adult_train, name) |> sort), nominal)

In [ ]:
# Collapse categories in 'native_country' column
adult_train.native_country[Not(isequal.(adult_train.native_country, "United-States"))] .= "not-United-States";

In [ ]:
# Check categories count after the operation
freqtable(adult_train.native_country)

In [ ]:
#Produce two-way table to learn more about relation between 'maritial_status' and 'relationship' features
@chain adult_train begin
    groupby([:marital_status, :relationship])
    combine(nrow)
    unstack(:marital_status, :relationship, :nrow)
end

In [ ]:
# Extract continuous features
continuous = names(adult_train, Int)

In [ ]:
# Define a helper function for drawing of a single histogram
histogram_helper(column_name) =
    histogram(adult_train[!, column_name], xlabel=column_name, legend=false)

# Compose a grid of histograms in a single plot
foreach(x -> display(histogram_helper(x)), continuous)

In [ ]:
# Check how many 'capital_gain' observations are equal to 0
mean(adult_train.capital_gain .== 0)

In [ ]:
# Check how many 'capital_loss' observations are equal to 0
mean(adult_train.capital_loss .== 0)

In [ ]:
# Check the same mean as above, but grouped by `target`
# It seems that wealthy people more often gain money on capital market, but also lose money more often (which seems reasonable)
@chain adult_train begin
    groupby(:target)
    combine([:capital_gain, :capital_loss] .=>
            (x -> mean(x .== 0)) .=>
            [:freq0_capital_gain, :freq0_capital_loss])
end

In [ ]:
# Save clean dataset as Arrow file
Arrow.write("adult_train.arrow", adult_train)